ENERGY CONSUMPTION OF NETHERLANDS

We will look at the electricity consumption in Netherlands for past 10 years, from 2010 to 2019. The three major network administrators in Netherlands those are Enexis, Liander, and Stedin together they provide energy to the entire country.

The main aim is to analyze the amount of electricity consumed individually on each connection, a comparison between consumption and delivery percentage in certain areas, and which network providers are responsible for that.

Smart meters spread across the cities, so an analysis is done to see cities that have the highest number of smart meters installed and what was their journey to reach this level of smart meters installation.

Enexis, Liander, and Stedin as they are the main energy distributors in Netherlands, therefore will analyze which cities receives the minimum or maximum amount of electricity and is it related to the number of electricity connections present?

In [11]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

DataFrame Setup

In [12]:
EnergyDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("electricity/*.csv")

Print Schema, This DataFrame has 1710916 rows

In [13]:
from IPython.display import display, Markdown

EnergyDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % EnergyDF.count()))

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: string (nullable = true)
 |-- delivery_perc: string (nullable = true)
 |-- perc_of_active_connections: string (nullable = true)
 |-- type_conn_perc: string (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: string (nullable = true)
 |-- annual_consume_lowtarif_perc: string (nullable = true)
 |-- smartmeter_perc: string (nullable = true)



This DataFrame has **1710916 rows**.

In [7]:
EnergyDF.show(3)

+-----------+-------------+--------------------+------------+----------+---------+---------------+-------------+--------------------------+--------------+------------------+--------------+----------------------------+---------------+
|net_manager|purchase_area|              street|zipcode_from|zipcode_to|     city|num_connections|delivery_perc|perc_of_active_connections|type_conn_perc|type_of_connection|annual_consume|annual_consume_lowtarif_perc|smartmeter_perc|
+-----------+-------------+--------------------+------------+----------+---------+---------------+-------------+--------------------------+--------------+------------------+--------------+----------------------------+---------------+
| Liander NB|      LIANDER|De Ruyterkade Ste...|      1011AA|    1011AB|AMSTERDAM|           36.0|        100.0|                     38.89|         100.0|              3x25|       16589.0|                       44.44|           25.0|
| Liander NB|      LIANDER|       De Ruyterkade|      1011AC|   

In [14]:
EnergyDF.cache()
EnergyDF.sample(False, 0.1).take(2)

[Row(net_manager='Liander NB', purchase_area='LIANDER', street='Prins Hendrikkade', zipcode_from='1011AH', zipcode_to='1011AH', city='AMSTERDAM', num_connections='21.0', delivery_perc='100.0', perc_of_active_connections='100.0', type_conn_perc='100.0', type_of_connection='1x25', annual_consume='4171.0', annual_consume_lowtarif_perc='42.86', smartmeter_perc='33.33'),
 Row(net_manager='Liander NB', purchase_area='LIANDER', street='Houtkopersdwarsstraat', zipcode_from='1011AL', zipcode_to='1011AM', city='AMSTERDAM', num_connections='16.0', delivery_perc='100.0', perc_of_active_connections='81.25', type_conn_perc='100.0', type_of_connection='3x25', annual_consume='12927.0', annual_consume_lowtarif_perc='68.75', smartmeter_perc='56.25')]

What the consumption of electricity per user?

In [15]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F
consumption_per_connection = EnergyDF.withColumn("consumption_per_connection", F.col("annual_consume") / F.col("num_connections")).select("consumption_per_connection", "annual_consume", "num_connections")

consumption_per_connection.printSchema()

consumption_per_connection.show(5)
                

root
 |-- consumption_per_connection: double (nullable = true)
 |-- annual_consume: string (nullable = true)
 |-- num_connections: string (nullable = true)

+--------------------------+--------------+---------------+
|consumption_per_connection|annual_consume|num_connections|
+--------------------------+--------------+---------------+
|        460.80555555555554|       16589.0|           36.0|
|         583.4615384615385|       22755.0|           39.0|
|        211.89473684210526|        4026.0|           19.0|
|        167.57894736842104|        3184.0|           19.0|
|                    1559.0|       21826.0|           14.0|
+--------------------------+--------------+---------------+
only showing top 5 rows



Maximum Consumption by a user?

In [16]:
row1 = consumption_per_connection.agg({"consumption_per_connection":"max"}).show()

+-------------------------------+
|max(consumption_per_connection)|
+-------------------------------+
|              9229.666666666666|
+-------------------------------+



Top 10 cities that have the highest amount of electricity consumption?

In [17]:
import pyspark.sql.functions as F
group_city = EnergyDF.groupBy("city")
group_city.agg(F.max("annual_consume").alias("top_ten_consumptions")).orderBy(F.max("annual_consume").desc()).show(10)

+----------+--------------------+
|      city|top_ten_consumptions|
+----------+--------------------+
|    MEPPEL|             9999.85|
|      BEST|             9999.78|
|ROOSENDAAL|             9999.66|
|  ROSMALEN|             9999.57|
|  RADEWIJK|             9999.45|
|       OSS|             9999.29|
|    BAEXEM|              9999.0|
| HEEMSKERK|              9999.0|
|   TER AAR|                9999|
|OKKENBROEK|                9999|
+----------+--------------------+
only showing top 10 rows



Which areas has the highest number of smart meters ?

Amsterdam has the highest percentage of smart meters


In [18]:
import pyspark.sql.functions as F
group_city_street = EnergyDF.groupBy("city" , "street")
group_city_street.agg(F.max("smartmeter_perc").alias("top_ten_areas")).orderBy(F.max("smartmeter_perc").desc()).show(10)

+-------------+--------------------+-------------+
|         city|              street|top_ten_areas|
+-------------+--------------------+-------------+
|    AMSTERDAM|Derde Amstelvlietpad|        99.74|
|    EINDHOVEN|          Torenallee|        99.67|
|    AMSTERDAM|   Bert Haanstrakade|        99.67|
|    AMSTERDAM|Mary van der Slui...|         99.6|
|   AMSTELVEEN|      Maimonideslaan|        99.56|
|'S-GRAVENHAGE|       Waldorpstraat|        99.55|
|       DIEMEN| Carel Willinkgracht|        99.53|
|       ALMERE|          Zeeduinweg|        99.49|
|      UTRECHT| Louis Armstronglaan|        99.47|
|      UTRECHT|       Parkzichtlaan|        99.46|
+-------------+--------------------+-------------+
only showing top 10 rows



One city that has maximum number of electricity connections present and one that consume the highest amount of electricity, are they related ? 

Highest annual consume is 9999.85 kwh

In [19]:
row1 = EnergyDF.agg({"annual_consume":"max"}).show()

+-------------------+
|max(annual_consume)|
+-------------------+
|            9999.85|
+-------------------+



Meppel city consumes highest amount of electricity

In [20]:
EnergyDF.where(EnergyDF.annual_consume == 9999.85)\
.select("city").collect()

[Row(city='MEPPEL')]

991 is the highest number of electricity connections present in Netherlands

In [21]:
row2 = EnergyDF.agg({"num_connections":"max"}).show()

+--------------------+
|max(num_connections)|
+--------------------+
|                 991|
+--------------------+



Gerkesklooster has 991 connections

In [22]:
EnergyDF.where(EnergyDF.num_connections == 991)\
.select("city").collect()

[Row(city='GERKESKLOOSTER'), Row(city='GERKESKLOOSTER')]

Compare which energy provider  has the maximum connections present and delivers the highest amount of electricity
and which has the minimum? 


In [23]:
group_company= EnergyDF.groupBy("net_manager")
group_company.agg(F.min("num_connections").alias("min_connections"),\
                F.max("num_connections").alias("max_connections"),\
                F.avg("num_connections").alias("avg_no_connections"),\
                 F.min("delivery_perc").alias("min_delivery_by"),\
                 F.max("delivery_perc").alias("max_delivery_by")).show()

+-----------------+---------------+---------------+------------------+---------------+---------------+
|      net_manager|min_connections|max_connections|avg_no_connections|min_delivery_by|max_delivery_by|
+-----------------+---------------+---------------+------------------+---------------+---------------+
|    8716946000005|             10|             88|19.857443609022557|          100.0|          99.21|
|    8716886000004|             10|             99| 23.36839298597823|            0.0|           99.1|
|    8716921000006|             10|             99| 23.24965021919597|            0.0|          99.46|
|    8716924000003|             10|             99|24.711040113596024|          100.0|          99.32|
|    8716874000009|             10|             99|23.079109538090503|            0.0|          99.83|
|    8716925000002|             10|             99|25.629728020240353|          10.71|          99.61|
|    8716892000005|             10|             99|26.913580744743292|   

In [24]:
x =group_company.agg(F.min("num_connections").alias("min_connections"),\
                F.max("num_connections").alias("max_connections"),\
                F.avg("num_connections").alias("avg_no_connections"))
x.agg(F.max("max_connections"), F.min("min_connections")).show()


+--------------------+--------------------+
|max(max_connections)|min(min_connections)|
+--------------------+--------------------+
|                 991|                   0|
+--------------------+--------------------+



99.84 % is the maximum electricity delivered in Neterlands  

In [25]:
y =group_company.agg(F.min("delivery_perc").alias("min_delivery_by"),\
                F.max("delivery_perc").alias("max_delivery_by"))
y.agg(F.max("max_delivery_by"), F.min("min_delivery_by")).show()


+--------------------+--------------------+
|max(max_delivery_by)|min(min_delivery_by)|
+--------------------+--------------------+
|               99.84|                 0.0|
+--------------------+--------------------+



Liander N.V. has 991 (maximum) connections, highest among all three network providers.

In [26]:
x.where(x.max_connections == 991 )\
.select("net_manager").show()

+-----------------+
|      net_manager|
+-----------------+
|Liander N.V. (ZW)|
+-----------------+



Enexis and Enexis B.V. has lowest number of connection in Netherlands

In [27]:
x.where(x.min_connections == 0 )\
.select("net_manager").show()

+-----------+
|net_manager|
+-----------+
|     Enexis|
|Enexis B.V.|
+-----------+



Liander N.V and Liander NB provides the highest amount of electricity in Netherlands

In [28]:
y.where(y.max_delivery_by == 99.84 )\
.select("net_manager").show()

+-----------------+
|      net_manager|
+-----------------+
|       Liander NB|
|Liander N.V. (NW)|
+-----------------+



In [29]:
y.where(y.min_delivery_by == 0.0 )\
.select("net_manager").show()

+-----------------+
|      net_manager|
+-----------------+
|    8716886000004|
|    8716921000006|
|    8716874000009|
|    8716892000005|
|       Liander NB|
|Liander N.V. (NW)|
|     Liander N.V.|
+-----------------+

